In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn import linear_model

In [2]:
## Store the usefull information from this huge dataset
dataset = {}
dataset["review_text"] = []
dataset['rating'] = []
count = 0
with open ('Books_5.json') as Train_json:
    for i in Train_json:
        count+=1
        if count % 20 == 0:
            item = json.loads(i)
            dataset["review_text"].append(item["reviewText"])
            dataset["rating"].append(item["overall"])

In [3]:
## Convert Dataset to dataframe
dataset_df = pd.DataFrame(dataset)
print(len(dataset_df))
dataset_df.head()

444902


,rating,review_text
0,5.0,"This book is everything that is simple, delica..."
1,5.0,When I first started writing poetry at age 12 ...
2,5.0,"Khalil Gibran's book, The Prophet, has the pow..."
3,5.0,I was given this book by a writer friend who c...
4,5.0,A book to be treasured. A tremendous poet deal...


In [4]:
dataset_df_5 = dataset_df[dataset_df["rating"]==5.0]
dataset_df_4 = dataset_df[dataset_df["rating"]==4.0]
dataset_df_3 = dataset_df[dataset_df["rating"]==3.0]
dataset_df_2 = dataset_df[dataset_df["rating"]==2.0]
dataset_df_1 = dataset_df[dataset_df["rating"]==1.0]

In [5]:
dataset_df_1_train = dataset_df_1.iloc[0:5000,]
dataset_df_1_test = dataset_df_1.iloc[5000:6000,]
dataset_df_2_train = dataset_df_2.iloc[0:5000,]
dataset_df_2_test = dataset_df_2.iloc[5000:6000,]
dataset_df_3_train = dataset_df_3.iloc[0:5000,]
dataset_df_3_test = dataset_df_3.iloc[5000:6000,]
dataset_df_4_train = dataset_df_4.iloc[0:5000,]
dataset_df_4_test = dataset_df_4.iloc[5000:6000,]
dataset_df_5_train = dataset_df_5.iloc[0:5000,]
dataset_df_5_test = dataset_df_5.iloc[5000:6000,]

In [6]:
frames_train = [dataset_df_5_train, dataset_df_4_train, dataset_df_3_train,dataset_df_2_train,dataset_df_1_train]
frames_test = [dataset_df_5_test, dataset_df_4_test, dataset_df_3_test,dataset_df_2_test,dataset_df_1_test]

dataset_df_train = pd.concat(frames_train)
dataset_df_test = pd.concat(frames_test)

In [7]:
## Clean the reviews
def data_cleaning (raw_review_str):
    ## remove potential HTML tag
    review_text = BeautifulSoup(raw_review_str).get_text()
    ## remove non-letter 
    review_letter = re.sub("[^a-zA-Z]"," ",review_text)
    ## convert to lower case and split the list
    review_lower_list = review_letter.lower().split()
    ## remove stop words and stemming
    stops_eng = set(stopwords.words("english"))
    stemmer = nltk.wordnet.WordNetLemmatizer()
    #review_nostopwords_stemmed = [stemmer.lemmatize(w) for w in review_lower_list]
    review_nostopwords_stemmed = [stemmer.lemmatize(w) for w in review_lower_list if w not in stops_eng]
    ## return 
    return " ".join(review_nostopwords_stemmed)

In [8]:
dataset_df_train["review_cleaned"] = dataset_df_train["review_text"].apply(data_cleaning)
dataset_df_test["review_cleaned"] = dataset_df_test["review_text"].apply(data_cleaning)

/Users/xinmu/anaconda/envs/py36/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/xinmu/anaconda/envs/py36/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html.parser")

  markup_type=markup_type))


In [9]:
## Put all the review of train_set in a list to generate the features
clean_train_reviews = []
for i in dataset_df_train["review_cleaned"]:
    clean_train_reviews.append(i)

clean_test_reviews = []
for i in dataset_df_test["review_cleaned"]:
    clean_test_reviews.append(i) 

In [12]:
## Generate Features (bag of words)
vectorizer = TfidfVectorizer(analyzer = "word", ngram_range=(1, 2), tokenizer = None, preprocessor = None, stop_words = None,max_features=2000)
train_data_features = vectorizer.fit_transform(clean_train_reviews)
test_data_features = vectorizer.transform(clean_test_reviews)

In [13]:
train_data_features.shape

(25000, 2000)

In [14]:
test_data_features.shape

(5000, 2000)

In [15]:
train_data_features = train_data_features.toarray()
test_data_features = test_data_features.toarray()

In [16]:
clf = LDA()
clf.fit(train_data_features, dataset_df_train["rating"])

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [17]:
prediction_lda = clf.predict(test_data_features)

In [18]:
print("prediction accuracy is %f" % ((dataset_df_test['rating'] == prediction_lda).sum()/len(dataset_df_test)))

prediction accuracy is 0.452000


In [19]:
qda = QDA()
qda.fit(train_data_features, dataset_df_train["rating"])

/Users/xinmu/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariance=False, store_covariances=None, tol=0.0001)

In [20]:
prediction_qda = qda.predict(test_data_features)

In [21]:
print("prediction accuracy is %f" % ((dataset_df_test['rating'] == prediction_qda).sum()/len(dataset_df_test)))

prediction accuracy is 0.388600


In [22]:
logreg = linear_model.LogisticRegression(C=1e5)
logreg.fit(train_data_features, dataset_df_train["rating"])

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [23]:
prediction_logreg = logreg.predict(test_data_features)

In [24]:
print("prediction accuracy is %f" % ((dataset_df_test['rating'] == prediction_logreg).sum()/len(dataset_df_test)))

prediction accuracy is 0.436400
